In [1]:
!pip install yfinance
!pip install PyPortfolioOpt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import yfinance as yf
import datetime

In [3]:
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

In [4]:
nifty_50_tickers=['ADANIENT.NS', 'ADANIPORTS.NS', 'APOLLOHOSP.NS', 'ASIANPAINT.NS', 'AXISBANK.NS', 'BAJAJ-AUTO.NS', 'BAJFINANCE.NS', 'BAJAJFINSV.NS', 'BPCL.NS', 'BHARTIARTL.NS', 'BRITANNIA.NS', 'CIPLA.NS', 'COALINDIA.NS', 'DIVISLAB.NS', 'DRREDDY.NS', 'EICHERMOT.NS', 'GRASIM.NS', 'HCLTECH.NS', 'HDFCBANK.NS', 'HDFCLIFE.NS', 'HEROMOTOCO.NS', 'HINDALCO.NS', 'HINDUNILVR.NS', 'HDFC.NS', 'ICICIBANK.NS', 'ITC.NS', 'INDUSINDBK.NS', 'INFY.NS', 'JSWSTEEL.NS', 'KOTAKBANK.NS', 'LT.NS', 'M&M.NS', 'MARUTI.NS', 'NTPC.NS', 'NESTLEIND.NS', 'ONGC.NS', 'POWERGRID.NS', 'RELIANCE.NS', 'SBILIFE.NS', 'SBIN.NS', 'SUNPHARMA.NS', 'TCS.NS', 'TATACONSUM.NS', 'TATAMOTORS.NS', 'TATASTEEL.NS', 'TECHM.NS', 'TITAN.NS', 'UPL.NS', 'ULTRACEMCO.NS', 'WIPRO.NS']

In [5]:
df=yf.download(nifty_50_tickers)['Close']
print(df)

[*********************100%***********************]  50 of 50 completed
            ADANIENT.NS  ADANIPORTS.NS  APOLLOHOSP.NS  ASIANPAINT.NS  \
Date                                                                   
1995-12-25          NaN            NaN            NaN            NaN   
1995-12-26          NaN            NaN            NaN            NaN   
1995-12-27          NaN            NaN            NaN            NaN   
1995-12-28          NaN            NaN            NaN            NaN   
1995-12-29          NaN            NaN            NaN            NaN   
...                 ...            ...            ...            ...   
2023-05-24  2475.600098     718.250000    4536.450195    3101.500000   
2023-05-25  2537.449951     724.500000    4568.149902    3123.550049   
2023-05-26  2544.350098     726.849976    4609.899902    3128.399902   
2023-05-29  2534.050049     737.500000    4640.450195    3137.399902   
2023-05-30  2496.449951     734.049988    4599.149902    3144.399

In [6]:
df = df.dropna()
df

,ADANIENT.NS,ADANIPORTS.NS,APOLLOHOSP.NS,ASIANPAINT.NS,AXISBANK.NS,BAJAJ-AUTO.NS,BAJAJFINSV.NS,BAJFINANCE.NS,BHARTIARTL.NS,BPCL.NS,...,SUNPHARMA.NS,TATACONSUM.NS,TATAMOTORS.NS,TATASTEEL.NS,TCS.NS,TECHM.NS,TITAN.NS,ULTRACEMCO.NS,UPL.NS,WIPRO.NS
Date,,,,,,,,,,,,,,,,,,,,,
2017-11-17,85.331535,400.950012,1054.849976,1171.300049,542.400024,3206.850098,512.974976,1787.349976,453.121033,504.000000,...,517.099976,259.899994,421.799988,66.805893,1353.650024,484.549988,786.299988,4267.049805,482.799988,221.362549
2017-11-20,87.402161,398.600006,1059.599976,1166.349976,545.150024,3242.000000,517.354980,1805.250000,452.340576,504.450012,...,517.150024,274.899994,423.000000,66.896400,1351.724976,476.799988,783.950012,4181.299805,485.933319,220.762558
2017-11-21,83.615089,400.799988,1107.800049,1160.300049,546.799988,3300.250000,520.619995,1809.750000,462.073273,511.500000,...,538.049988,273.149994,424.250000,67.772919,1336.474976,493.850006,815.450012,4163.399902,503.000000,220.500061
2017-11-22,85.985413,414.500000,1149.500000,1174.300049,540.500000,3313.199951,517.635010,1773.949951,458.262817,507.700012,...,535.250000,274.049988,428.600006,67.744339,1340.324951,482.600006,813.150024,4230.399902,498.066681,220.725052
2017-11-23,83.097435,407.100006,1139.550049,1159.599976,544.150024,3264.649902,513.359985,1763.650024,455.737823,505.149994,...,544.750000,274.799988,426.250000,67.901543,1340.625000,489.600006,805.900024,4185.149902,499.466675,220.162552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-24,2475.600098,718.250000,4536.450195,3101.500000,917.700012,4517.600098,1426.449951,6785.950195,801.400024,363.299988,...,952.150024,771.150024,520.200012,105.199997,3303.350098,1098.750000,2707.250000,7653.799805,682.250000,399.700012
2023-05-25,2537.449951,724.500000,4568.149902,3123.550049,921.049988,4644.000000,1427.349976,6841.149902,822.700012,363.299988,...,945.000000,780.299988,514.849976,104.849998,3293.500000,1098.099976,2707.949951,7621.750000,675.799988,394.100006
2023-05-26,2544.350098,726.849976,4609.899902,3128.399902,926.950012,4615.950195,1439.150024,6904.500000,817.950012,364.299988,...,969.900024,789.700012,518.450012,106.400002,3328.899902,1114.750000,2742.149902,7721.850098,688.099976,401.700012


In [7]:
df.describe()

,ADANIENT.NS,ADANIPORTS.NS,APOLLOHOSP.NS,ASIANPAINT.NS,AXISBANK.NS,BAJAJ-AUTO.NS,BAJAJFINSV.NS,BAJFINANCE.NS,BHARTIARTL.NS,BPCL.NS,...,SUNPHARMA.NS,TATACONSUM.NS,TATAMOTORS.NS,TATASTEEL.NS,TCS.NS,TECHM.NS,TITAN.NS,ULTRACEMCO.NS,UPL.NS,WIPRO.NS
count,1366.000000,1366.000000,1366.000000,1366.000000,1366.000000,1366.000000,1366.000000,1366.000000,1366.000000,1366.000000,...,1366.000000,1366.000000,1366.000000,1366.000000,1366.000000,1366.000000,1366.000000,1366.000000,1366.000000,1366.000000
mean,973.953086,529.558162,2582.775545,2180.307391,673.620242,3285.777819,998.547215,4483.572255,527.992180,389.968375,...,638.775183,499.868375,288.878111,75.353066,2607.446083,911.338543,1551.669948,5298.091584,599.181857,355.276724
std,1096.816489,193.649787,1469.481773,783.158475,134.262032,514.762222,429.286922,1928.977513,168.919018,59.483456,...,201.853355,232.439462,131.640612,33.870380,729.367960,293.557394,654.895055,1456.798821,133.256870,142.407373
min,67.402306,207.800003,927.099976,1101.599976,303.149994,1935.550049,409.725006,1596.449951,262.002197,257.200012,...,324.500000,179.750000,65.300003,25.375000,1272.800049,465.399994,749.500000,3018.100098,254.600006,162.350006
25%,140.899994,368.612503,1249.662506,1407.899994,568.574997,2868.937439,617.761261,2682.199951,360.812500,343.874992,...,465.137489,270.574989,170.649994,47.428749,2014.500000,695.337494,970.012497,4037.987549,488.474998,241.518764
50%,278.625000,410.175003,1726.449951,1944.774963,698.875000,3199.950073,890.482513,4153.750000,523.300018,380.549988,...,574.725006,490.899994,295.774994,62.330000,2282.099976,814.350006,1272.575012,4550.174805,586.225006,294.424988
75%,1696.125031,725.937485,4283.187622,2992.662476,755.837479,3715.700073,1366.487549,6180.737427,691.249985,429.824997,...,825.087479,739.337494,416.987503,108.074999,3265.262451,1057.199951,2309.562561,6729.599854,724.537521,426.612511
max,4165.299805,970.250000,5733.950195,3576.300049,962.299988,4644.000000,1907.655029,7929.299805,851.200012,544.599976,...,1051.949951,882.099976,530.849976,151.940002,4019.149902,1806.099976,2812.250000,8214.049805,853.549988,721.500000


#### Mean Variance Optimization

In [8]:
mu = mean_historical_return(df)
S = CovarianceShrinkage(df).ledoit_wolf()

In [9]:
ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()

cleaned_weights = ef.clean_weights()
print(dict(cleaned_weights))
ef.portfolio_performance(verbose=True)

{'ADANIENT.NS': 0.34278, 'ADANIPORTS.NS': 0.0, 'APOLLOHOSP.NS': 0.1247, 'ASIANPAINT.NS': 0.0, 'AXISBANK.NS': 0.0, 'BAJAJ-AUTO.NS': 0.0, 'BAJAJFINSV.NS': 0.0, 'BAJFINANCE.NS': 0.0, 'BHARTIARTL.NS': 0.0, 'BPCL.NS': 0.0, 'BRITANNIA.NS': 0.0, 'CIPLA.NS': 0.0, 'COALINDIA.NS': 0.0, 'DIVISLAB.NS': 0.10001, 'DRREDDY.NS': 0.0, 'EICHERMOT.NS': 0.0, 'GRASIM.NS': 0.0, 'HCLTECH.NS': 0.01312, 'HDFC.NS': 0.0, 'HDFCBANK.NS': 0.0, 'HDFCLIFE.NS': 0.0, 'HEROMOTOCO.NS': 0.0, 'HINDALCO.NS': 0.0, 'HINDUNILVR.NS': 0.0, 'ICICIBANK.NS': 0.0, 'INDUSINDBK.NS': 0.0, 'INFY.NS': 0.07532, 'ITC.NS': 0.0, 'JSWSTEEL.NS': 0.0, 'KOTAKBANK.NS': 0.0, 'LT.NS': 0.0, 'M&M.NS': 0.0, 'MARUTI.NS': 0.0, 'NESTLEIND.NS': 0.23367, 'NTPC.NS': 0.0, 'ONGC.NS': 0.0, 'POWERGRID.NS': 0.0, 'RELIANCE.NS': 0.0, 'SBILIFE.NS': 0.0, 'SBIN.NS': 0.0, 'SUNPHARMA.NS': 0.0, 'TATACONSUM.NS': 0.0, 'TATAMOTORS.NS': 0.0, 'TATASTEEL.NS': 0.0, 'TCS.NS': 0.01378, 'TECHM.NS': 0.0, 'TITAN.NS': 0.09662, 'ULTRACEMCO.NS': 0.0, 'UPL.NS': 0.0, 'WIPRO.NS': 0.0}
Ex

(0.45575658518168016, 0.2584319486036739, 1.6861560172266)

In [10]:
latest_prices = get_latest_prices(df)

da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=100000)

allocation, leftover = da.greedy_portfolio()
print("Discrete allocation:", allocation)
print("Funds remaining: Rs. {:.2f}".format(leftover))

Discrete allocation: {'ADANIENT.NS': 13, 'NESTLEIND.NS': 1, 'APOLLOHOSP.NS': 3, 'DIVISLAB.NS': 3, 'TITAN.NS': 3, 'INFY.NS': 6, 'TCS.NS': 1, 'HCLTECH.NS': 1}
Funds remaining: Rs. 997.60


#### SLSQP

In [11]:
from scipy.optimize import minimize
import numpy as np

In [12]:
close_prices=df
returns = close_prices.pct_change().dropna()

# Number of stocks in the portfolio
num_stocks = len(close_prices.columns)
days=1365

# Define the objective function for portfolio optimization
def objective(weights):
    portfolio_return = np.sum(returns.mean() * weights) * days  # trading days in a year
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(returns.cov() * days, weights)))
    return portfolio_volatility

# Define the constraints for portfolio optimization
constraints = [{'type': 'eq', 'fun': lambda x: np.sum(x) - 1}]

# Define the bounds for portfolio optimization
bounds = [(0, 1)] * num_stocks

# Set an initial set of equal weights
initial_weights = [1 / num_stocks] * num_stocks

# Optimize the portfolio
result = minimize(objective, initial_weights, method='SLSQP', bounds=bounds, constraints=constraints)

# Extract the optimized weights
optimized_weights = result.x

# Print the optimized weights
for i, stock in enumerate(close_prices.columns):
    print(f'{stock}: {optimized_weights[i]}')

ADANIENT.NS: 0.0
ADANIPORTS.NS: 1.2044685134997226e-17
APOLLOHOSP.NS: 7.471473013195924e-06
ASIANPAINT.NS: 0.04782358108161503
AXISBANK.NS: 0.0
BAJAJ-AUTO.NS: 0.030861405425726157
BAJAJFINSV.NS: 5.398870916993658e-18
BAJFINANCE.NS: 0.0
BHARTIARTL.NS: 0.016636953049643556
BPCL.NS: 3.190952833095781e-18
BRITANNIA.NS: 0.05500419545629558
CIPLA.NS: 0.08965439231517502
COALINDIA.NS: 0.04046858135343591
DIVISLAB.NS: 0.005117961025009475
DRREDDY.NS: 0.0834225660409384
EICHERMOT.NS: 0.0
GRASIM.NS: 0.0
HCLTECH.NS: 0.0
HDFC.NS: 1.1538009477154803e-17
HDFCBANK.NS: 0.06221052878974367
HDFCLIFE.NS: 0.0
HEROMOTOCO.NS: 0.0
HINDALCO.NS: 0.0
HINDUNILVR.NS: 0.05580017151063754
ICICIBANK.NS: 1.4811282049905356e-17
INDUSINDBK.NS: 0.0
INFY.NS: 0.01175710965193534
ITC.NS: 0.08443723314013926
JSWSTEEL.NS: 3.01020531105694e-17
KOTAKBANK.NS: 0.0
LT.NS: 0.0
M&M.NS: 0.0
MARUTI.NS: 0.0
NESTLEIND.NS: 0.1015300543560216
NTPC.NS: 0.03407129487904673
ONGC.NS: 0.0
POWERGRID.NS: 0.11565413141687701
RELIANCE.NS: 0.0
SBI

In [13]:
df.pct_change()

,ADANIENT.NS,ADANIPORTS.NS,APOLLOHOSP.NS,ASIANPAINT.NS,AXISBANK.NS,BAJAJ-AUTO.NS,BAJAJFINSV.NS,BAJFINANCE.NS,BHARTIARTL.NS,BPCL.NS,...,SUNPHARMA.NS,TATACONSUM.NS,TATAMOTORS.NS,TATASTEEL.NS,TCS.NS,TECHM.NS,TITAN.NS,ULTRACEMCO.NS,UPL.NS,WIPRO.NS
Date,,,,,,,,,,,,,,,,,,,,,
2017-11-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-11-20,0.024266,-0.005861,0.004503,-0.004226,0.005070,0.010961,0.008538,0.010015,-0.001722,0.000893,...,0.000097,0.057715,0.002845,0.001355,-0.001422,-0.015994,-0.002989,-0.020096,0.006490,-0.002710
2017-11-21,-0.043329,0.005519,0.045489,-0.005187,0.003027,0.017967,0.006311,0.002493,0.021516,0.013976,...,0.040414,-0.006366,0.002955,0.013103,-0.011282,0.035759,0.040181,-0.004281,0.035121,-0.001189
2017-11-22,0.028348,0.034182,0.037642,0.012066,-0.011522,0.003924,-0.005734,-0.019782,-0.008246,-0.007429,...,-0.005204,0.003295,0.010253,-0.000422,0.002881,-0.022780,-0.002821,0.016093,-0.009808,0.001020
2017-11-23,-0.033587,-0.017853,-0.008656,-0.012518,0.006753,-0.014654,-0.008259,-0.005806,-0.005510,-0.005023,...,0.017749,0.002737,-0.005483,0.002321,0.000224,0.014505,-0.008916,-0.010696,0.002811,-0.002548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-24,-0.060030,-0.021524,-0.003066,-0.006121,0.001692,0.004335,-0.007514,0.000796,0.001813,-0.009002,...,0.022059,0.001429,-0.015705,-0.003788,0.001911,0.007565,0.008869,-0.000059,0.002351,0.005155
2023-05-25,0.024984,0.008702,0.006988,0.007109,0.003650,0.027979,0.000631,0.008134,0.026578,0.000000,...,-0.007509,0.011865,-0.010285,-0.003327,-0.002982,-0.000592,0.000259,-0.004187,-0.009454,-0.014011
2023-05-26,0.002719,0.003244,0.009139,0.001553,0.006406,-0.006040,0.008267,0.009260,-0.005774,0.002753,...,0.026349,0.012047,0.006992,0.014783,0.010748,0.015163,0.012629,0.013133,0.018201,0.019284


In [14]:
optimized_weights = [round(1000*num)/10 for num in optimized_weights]
cost=[]
for i in range(50):
   cost.append(df.iloc[0,i]*optimized_weights[i])
sum(cost)

170426.41430969237

#### Hierarchical Risk Parity (HRP)

In [15]:
ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()

cleaned_weights = ef.clean_weights()
print(dict(cleaned_weights))

{'ADANIENT.NS': 0.34278, 'ADANIPORTS.NS': 0.0, 'APOLLOHOSP.NS': 0.1247, 'ASIANPAINT.NS': 0.0, 'AXISBANK.NS': 0.0, 'BAJAJ-AUTO.NS': 0.0, 'BAJAJFINSV.NS': 0.0, 'BAJFINANCE.NS': 0.0, 'BHARTIARTL.NS': 0.0, 'BPCL.NS': 0.0, 'BRITANNIA.NS': 0.0, 'CIPLA.NS': 0.0, 'COALINDIA.NS': 0.0, 'DIVISLAB.NS': 0.10001, 'DRREDDY.NS': 0.0, 'EICHERMOT.NS': 0.0, 'GRASIM.NS': 0.0, 'HCLTECH.NS': 0.01312, 'HDFC.NS': 0.0, 'HDFCBANK.NS': 0.0, 'HDFCLIFE.NS': 0.0, 'HEROMOTOCO.NS': 0.0, 'HINDALCO.NS': 0.0, 'HINDUNILVR.NS': 0.0, 'ICICIBANK.NS': 0.0, 'INDUSINDBK.NS': 0.0, 'INFY.NS': 0.07532, 'ITC.NS': 0.0, 'JSWSTEEL.NS': 0.0, 'KOTAKBANK.NS': 0.0, 'LT.NS': 0.0, 'M&M.NS': 0.0, 'MARUTI.NS': 0.0, 'NESTLEIND.NS': 0.23367, 'NTPC.NS': 0.0, 'ONGC.NS': 0.0, 'POWERGRID.NS': 0.0, 'RELIANCE.NS': 0.0, 'SBILIFE.NS': 0.0, 'SBIN.NS': 0.0, 'SUNPHARMA.NS': 0.0, 'TATACONSUM.NS': 0.0, 'TATAMOTORS.NS': 0.0, 'TATASTEEL.NS': 0.0, 'TCS.NS': 0.01378, 'TECHM.NS': 0.0, 'TITAN.NS': 0.09662, 'ULTRACEMCO.NS': 0.0, 'UPL.NS': 0.0, 'WIPRO.NS': 0.0}


In [16]:
ef.portfolio_performance(verbose=True)

Expected annual return: 45.6%
Annual volatility: 25.8%
Sharpe Ratio: 1.69


(0.45575658518168016, 0.2584319486036739, 1.6861560172266)

In [17]:
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

latest_prices = get_latest_prices(df)

da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=100000)

allocation, leftover = da.greedy_portfolio()
print("Discrete allocation:", allocation)
print("Funds remaining: Rs. {:.2f}".format(leftover))

Discrete allocation: {'ADANIENT.NS': 13, 'NESTLEIND.NS': 1, 'APOLLOHOSP.NS': 3, 'DIVISLAB.NS': 3, 'TITAN.NS': 3, 'INFY.NS': 6, 'TCS.NS': 1, 'HCLTECH.NS': 1}
Funds remaining: Rs. 997.60


#### Hierarchical Risk Parity (HRP)

In [18]:
from pypfopt import HRPOpt

In [19]:
hrp = HRPOpt(returns)
hrp_weights = hrp.optimize()

In [20]:
hrp.portfolio_performance(verbose=True)
print(dict(hrp_weights))

Expected annual return: 16.4%
Annual volatility: 17.1%
Sharpe Ratio: 0.84
{'ADANIENT.NS': 0.007168814738585328, 'ADANIPORTS.NS': 0.020711603182815758, 'APOLLOHOSP.NS': 0.021797034485586255, 'ASIANPAINT.NS': 0.026255746628966584, 'AXISBANK.NS': 0.007680060772425486, 'BAJAJ-AUTO.NS': 0.024842827609723468, 'BAJAJFINSV.NS': 0.008572585600015909, 'BAJFINANCE.NS': 0.007367642657465562, 'BHARTIARTL.NS': 0.021961756548785807, 'BPCL.NS': 0.015452340256559791, 'BRITANNIA.NS': 0.0271456092384923, 'CIPLA.NS': 0.024026874644259552, 'COALINDIA.NS': 0.02923797514515465, 'DIVISLAB.NS': 0.02020997259102947, 'DRREDDY.NS': 0.02532328631612724, 'EICHERMOT.NS': 0.014647524133859167, 'GRASIM.NS': 0.01846005071247866, 'HCLTECH.NS': 0.016864126150113373, 'HDFC.NS': 0.013007074539660319, 'HDFCBANK.NS': 0.017988495409805558, 'HDFCLIFE.NS': 0.026461576289795702, 'HEROMOTOCO.NS': 0.027688815016615628, 'HINDALCO.NS': 0.010806437652930037, 'HINDUNILVR.NS': 0.029661613826854876, 'ICICIBANK.NS': 0.009177581153519404,

In [21]:
hrp.portfolio_performance(verbose=True)
print(dict(hrp_weights))

Expected annual return: 16.4%
Annual volatility: 17.1%
Sharpe Ratio: 0.84
{'ADANIENT.NS': 0.007168814738585328, 'ADANIPORTS.NS': 0.020711603182815758, 'APOLLOHOSP.NS': 0.021797034485586255, 'ASIANPAINT.NS': 0.026255746628966584, 'AXISBANK.NS': 0.007680060772425486, 'BAJAJ-AUTO.NS': 0.024842827609723468, 'BAJAJFINSV.NS': 0.008572585600015909, 'BAJFINANCE.NS': 0.007367642657465562, 'BHARTIARTL.NS': 0.021961756548785807, 'BPCL.NS': 0.015452340256559791, 'BRITANNIA.NS': 0.0271456092384923, 'CIPLA.NS': 0.024026874644259552, 'COALINDIA.NS': 0.02923797514515465, 'DIVISLAB.NS': 0.02020997259102947, 'DRREDDY.NS': 0.02532328631612724, 'EICHERMOT.NS': 0.014647524133859167, 'GRASIM.NS': 0.01846005071247866, 'HCLTECH.NS': 0.016864126150113373, 'HDFC.NS': 0.013007074539660319, 'HDFCBANK.NS': 0.017988495409805558, 'HDFCLIFE.NS': 0.026461576289795702, 'HEROMOTOCO.NS': 0.027688815016615628, 'HINDALCO.NS': 0.010806437652930037, 'HINDUNILVR.NS': 0.029661613826854876, 'ICICIBANK.NS': 0.009177581153519404,

#### Mean Conditional Value at Risk (mCVAR)

In [22]:
from pypfopt.efficient_frontier import EfficientCVaR

In [23]:
S = df.cov()
ef_cvar = EfficientCVaR(mu, S)
cvar_weights = ef_cvar.min_cvar()

cleaned_weights = ef_cvar.clean_weights()
print(dict(cleaned_weights))

{'ADANIENT.NS': 0.0, 'ADANIPORTS.NS': 0.0, 'APOLLOHOSP.NS': 0.0, 'ASIANPAINT.NS': 0.0, 'AXISBANK.NS': 0.0, 'BAJAJ-AUTO.NS': 0.0, 'BAJAJFINSV.NS': 0.0, 'BAJFINANCE.NS': 0.0, 'BHARTIARTL.NS': 0.0, 'BPCL.NS': 0.0, 'BRITANNIA.NS': 0.0, 'CIPLA.NS': 0.0, 'COALINDIA.NS': 0.0, 'DIVISLAB.NS': 0.0, 'DRREDDY.NS': 0.0, 'EICHERMOT.NS': 0.0, 'GRASIM.NS': 0.0, 'HCLTECH.NS': 0.0, 'HDFC.NS': 0.0, 'HDFCBANK.NS': 0.0, 'HDFCLIFE.NS': 0.0, 'HEROMOTOCO.NS': 0.0, 'HINDALCO.NS': 0.0, 'HINDUNILVR.NS': 0.0, 'ICICIBANK.NS': 0.0, 'INDUSINDBK.NS': 0.0, 'INFY.NS': 0.0, 'ITC.NS': 0.0, 'JSWSTEEL.NS': 0.0, 'KOTAKBANK.NS': 0.0, 'LT.NS': 0.0, 'M&M.NS': 0.0, 'MARUTI.NS': 0.70756, 'NESTLEIND.NS': 0.0, 'NTPC.NS': 0.0, 'ONGC.NS': 0.0, 'POWERGRID.NS': 0.0, 'RELIANCE.NS': 0.0, 'SBILIFE.NS': 0.0, 'SBIN.NS': 0.0, 'SUNPHARMA.NS': 0.0, 'TATACONSUM.NS': 0.0, 'TATAMOTORS.NS': 0.0, 'TATASTEEL.NS': 0.0, 'TCS.NS': 0.0, 'TECHM.NS': 0.0, 'TITAN.NS': 0.0, 'ULTRACEMCO.NS': 0.29244, 'UPL.NS': 0.0, 'WIPRO.NS': 0.0}


In [24]:
da_cvar = DiscreteAllocation(cvar_weights, latest_prices, total_portfolio_value=100000)

allocation, leftover = da_cvar.greedy_portfolio()
print("Discrete allocation (CVAR):", allocation)
print("Funds remaining (CVAR): ${:.2f}".format(leftover))

Discrete allocation (CVAR): {'MARUTI.NS': 7, 'ULTRACEMCO.NS': 4, 'INDUSINDBK.NS': 1}
Funds remaining (CVAR): $1415.50


/usr/local/lib/python3.10/dist-packages/pypfopt/discrete_allocation.py:212: RuntimeWarning: invalid value encountered in true_divide
  current_weights /= current_weights.sum()
